# Introduction to pyodbc for Cloud SQL for SQL Server
This notebook covers how to get started with pyodbc, including how to connect to a SQL Server instance in Cloud SQL and examples of how to handle some common scenarios for developers and data scientists. You can download a copy of this notebook from the GitHub repo https://github.com/dmahugh/cloud-sql-pyodbc, and more information is available in the blog post [Getting started with pyodbc and Cloud SQL for SQL Server](https://medium.com/@dmahugh_70618/using-pyodbc-with-cloud-sql-for-sql-server-602fb7a1be1b).

The snippets below connect to Cloud SQL via the Cloud SQL Proxy listening on 127.0.0.1, so the notebook needs to run on your local machine. Connecting via the proxy is the recommended best practice for local development of Cloud SQL applications.

I've tested this notebook on a Windows 10 laptop in [VS Code](https://code.visualstudio.com/docs/python/jupyter-support), [Azure Data Studio](https://docs.microsoft.com/en-us/sql/azure-data-studio/download?view=sql-server-ver15). Anaconda is the "strongly recommended" option on the [Jupyter.org install page](https://jupyter.org/install), so start there if you've never worked with notebooks locally.

## Setup
Before you can run the code snippets in this notebook, you'll need to go through these one-time setup steps:

* Create a Cloud SQL for SQL Server instance and service account, then configure a local proxy connection as covered in [Cloud SQL Setup](https://medium.com/@dmahugh_70618/cloud-sql-setup-4fc72d3f33db).
* Clone the [GitHub repo](https://github.com/dmahugh/cloud-sql-pyodbc) to a local folder, and edit the ```config.py``` file to enter your project and Cloud SQL settings. (See the **ODBC Drivers** section below for information about how to set the ```driver``` setting.)
* Install the ```pyodbc``` package in your notebook environment. The way to do this can vary depending on which notebook software you're running, but in most cases you can simply do a ```!pip install pyodbc``` command in a notebook cell.

## ODBC drivers
The pyodbc module communicates with your database through an _ODBC driver_. [ODBC (Open Database Connectivity)](https://en.wikipedia.org/wiki/Open_Database_Connectivity) is an API standard developed in the early 90s by Microsoft and others, with the goal of providing a consistent programming interface for database access that could be used on any platform and from any programming language.

The recommended driver for Cloud SQL for SQL Server is [ODBC Driver 17 for SQL Server](https://www.microsoft.com/en-us/download/details.aspx?id=56567). It provides full support for all the features of the SQL Server version currently available on Cloud SQL, SQL Server 2017. For this notebook, you can also use the ```SQL Server Native Client``` driver or even the ```SQL Server``` driver released in 2000 and still found on many Windows machines, but those are deprecated, don't support all of SQL Server 2017's features, and shouldn't be used in production.

If you're using Linux or Mac OS for local development, see [Installing the Microsoft ODBC Driver for SQL Server on Linux and macOS](https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15) to get an ODBC driver for your environment. There are also some good tips and additional information in the [Connecting to SQL Server from Linux](https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Linux) article on the pyodbc wiki.


In [2]:
# Example of how to check which ODBC drivers are installed on your system.
import pyodbc

for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
ODBC Driver 17 for SQL Server
MySQL ODBC 8.0 ANSI Driver
MySQL ODBC 8.0 Unicode Driver


ODBC drivers are referenced by name, so copy one of the above names to the ```driver=``` setting in your config.py file. Ideally you'll be using **ODBC Driver 17 for SQL Server** if you have it. This notebook also works fine with the **SQL Server Native Client 11.0** driver, although that one's deprecated and shouldn't be used in production.

## Connection string syntax
The code snippet below shows how to construct a Cloud SQL for SQL Server connection string from the settings in ```config.py```.

In [3]:
# Example of creating a connection string from settings stored in config.py.

import config

CONNECTION_STRING = f"DRIVER={{{config.driver}}};SERVER={config.server};UID={config.sql_user};PWD={config.sql_password}"

print(" config.py settings: ".center(80, "-"))
print(f"      driver = {config.driver}")
print(f"      server = {config.server}")
print(f"    sql_user = {config.sql_user}")
print(f"sql_password = <{len(config.sql_password)} characters>")
print(" CONNECTION_STRING: ".center(80, "-"))
before, after = CONNECTION_STRING.split("PWD=")
print(before + f"PWD=<{len(after)} characters>")

----------------------------- config.py settings: ------------------------------
      driver = ODBC Driver 17 for SQL Server
      server = 127.0.0.1
    sql_user = sqlserver
sql_password = <16 characters>
------------------------------ CONNECTION_STRING: ------------------------------
DRIVER={ODBC Driver 17 for SQL Server};SERVER=127.0.0.1;UID=sqlserver;PWD=<16 characters>


Here are a few things to note about the connection string we've created:

* This connection string doesn't include a ```DATABASE=``` parameter. That's because we're going to connect to a newly created Cloud SQL instance that may not have any databases in it yet. Once a database has been created within the instance, either programmatically as shown below or manually in the Cloud Console, you can include the DATABASE parameter and then you don't need to ```USE <database>``` in your code.
* We're not using the ```Trusted_Connection=``` parameter either. That's for Windows authentication, and it allows you to omit the ```UID``` and ```PWD``` parameters to authenticate to SQL Server with your current Windows account. (This is not supported in Cloud SQL yet, but it's coming.)
* The triple braces around ```config.driver``` are necessary because the driver name needs to have braces around it in the connection string. So there are braces around config.driver to embed the driver name, plus double braces around _that_ to put braces around the driver name. (A double brace is how you escape a brace in a Python f-string.)


## Reminder: is the Cloud SQL Proxy running?
Now that we have a valid connection string, we're ready to connect to Cloud SQL and execute some SQL commands! And that means the Cloud SQL Proxy needs to be running locally and listening for connections, as shown in the diagram below. If you have questions about how to do that, see [Cloud SQL Setup](https://medium.com/@dmahugh_70618/cloud-sql-setup-4fc72d3f33db).

![using the Cloud SQL Proxy](https://github.com/dmahugh/cloud-sql-pyodbc/raw/master/images/notebook_proxy.png)

When the proxy is running and connected to Cloud SQL, you'll see this on the console:
```
YYYY/MM/DD HH:MM:SS using credential file for authentication; email=<SERVICE-ACCOUNT>@<PROJECT>.iam.gserviceaccount.com
YYYY/MM/DD HH:MM:SS Listening on 127.0.0.1:1433 for <PROJECT>:<REGION>:<INSTANCE>
YYYY/MM/DD HH:MM:SS Ready for new connections
```


## Creating a database
The first thing we'll do is create a database named ```testdb```, which we'll use for the other snippets to follow.

A common issue in creating SQL Server resources such as databases, tables, or users is that you may not know whether that resource already exists. For example, if you have already creates the ```testdb``` database and then try to execute ```CREATE DATABASE testdb``` again, you'll get an error because the database already exists.

The idiomatic way to get around that issue varies between database engines and versions. You need to use a different syntax for MySQL than for SQL Server, and older versions of SQL Server require yet another approach.

The snippet below demonstrates the recommended approach for creating a database if it doesn't already exist in SQL Server 2016 and above, using the ```DROP DATABASE IF EXISTS``` command.

In [4]:
# Example of how to create an empty database. Any existing database of the same name is dropped first.
import pyodbc

database = "testdb"
with pyodbc.connect(CONNECTION_STRING, autocommit=True).cursor() as cursor:
    cursor.execute(f"DROP DATABASE IF EXISTS {database}")
    cursor.execute(f"CREATE DATABASE {database}")

    # Print the list of databases in the instance, showing that testdb has been created.
    cursor.execute("SELECT name FROM master.sys.databases")
    for row in cursor:
        db_name = row[0]
        print(db_name + (" <<--- CREATED" if db_name == database else ""))


master
tempdb
model
msdb
testdb <<--- CREATED


A couple more details to note about the above code:
* The ```with``` statement creates a cursor from the pyodbc connection as a _context manager_, which is the Python construct for managing limited resources (such as database connections) that need to be allocated and then released. We don't need to explicitly close the connection or cursor, because that happens automatically at the end of the indented ```with``` block.
* The ```autocommit``` setting determines how database transactions are handled. With ```autocommit=True```, we're foregoing the ability to commit or rollback a multi-statement transaction, and instead each SQL command passed to ```cursor.execute()``` is committed to the database immediately. You _must_ do this for a ```CREATE DATABASE``` command, which is not allowed within a multi-statement transaction.


## Getting some sample data
Let's get some data to use for exploring a few scenarios. The code below downloads a copy of all active [dog licenses in New York City](https://data.cityofnewyork.us/api/views/nu7n-tubp/rows.csv?accessType=DOWNLOAD) and saves them as a CSV file.

In [6]:
import pathlib
import requests

filename = "NYC_Dog_Licensing_Dataset.csv"
dataset = "https://data.cityofnewyork.us/api/views/nu7n-tubp/rows.csv?accessType=DOWNLOAD"

if pathlib.Path(filename).is_file():
    print(f"{filename} already exists, not downloaded again.")
else:
    response = requests.get(dataset)
    with open(filename, "w") as csvfile:
        csvfile.write(response.text)
    print(f"{filename} downloaded, {len(response.text)} total bytes")

NYC_Dog_Licensing_Dataset.csv downloaded, 24083896 total bytes


Let's take a peek at the data to see how it's structured. A simple way to do this with a CSV file is to read it into a Pandas dataframe and then use the ```head()``` method to display the first 5 rows.

In [17]:
import pandas as pd

df = pd.read_csv(filename, encoding="ISO-8859-1")
print(df.head())

   RowNumber AnimalName AnimalGender  AnimalBirthMonth  \
0          1      PAIGE            F              2014   
1          2       YOGI            M              2010   
2          3        ALI            M              2014   
3          4      QUEEN            F              2013   
4          5       LOLA            F              2009   

                              BreedName  Borough  ZipCode LicenseIssuedDate  \
0  American Pit Bull Mix / Pit Bull Mix      NaN    10035        09/12/2014   
1                                 Boxer      NaN    10465        09/12/2014   
2                               Basenji      NaN    10013        09/12/2014   
3                      Akita Crossbreed      NaN    10013        09/12/2014   
4                               Maltese      NaN    10028        09/12/2014   

  LicenseExpiredDate  Extract Year  
0         09/12/2017          2016  
1         10/02/2017          2016  
2         09/12/2019          2016  
3         09/12/2017        

## Creating a table
Next we'll create a ```samoyed``` table, where we'll store some information about Samoyeds in the dog license data set.

In [12]:
CONNECTION_STRING = f"DRIVER={{{config.driver}}};SERVER={config.server};DATABASE={database};UID={config.sql_user};PWD={config.sql_password}"

table = "samoyed"
with pyodbc.connect(CONNECTION_STRING, autocommit=True).cursor() as cursor:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    cursor.execute(f"""CREATE TABLE {table} (
        name VARCHAR(50),
        gender CHAR(1),
        yearborn CHAR(4),
        zipcode CHAR(5),
        licensed DATE)""")

    # To confirm creation, print out all tables in our new database that are in the default dbo schema.
    for dbname, schema, tablename, *_ in cursor.tables():
        if dbname == database and schema == "dbo":
            print(tablename + (" <<--- CREATED" if tablename == table else ""))

samoyed <<--- CREATED


Note that the connection string in the snippet above has a ```DATABASE=``` parameter to specify the new database we created above. If we used a connection string without that parameter, we'd be connecting to the SQL Server _instance_ in Cloud SQL and then our T-SQL code would need to always include a ```USE <dtabase>``` command to open the proper database.

## Adding rows to the table
We've downloaded a dataset (CSV file) and created a table in Cloud SQL, next we'll use the ```csv``` module from the Python standard to iterate through the CSV file and insert records into our table. We'll just grab the first 20 Samoyed dogs registered in NYC and put them in our ```samoyed``` table.

Our goal here is to learn a few things about working with pyodbc, so we're writing our own code to handle data migration. But you don't need to write your own migration code if you want to migrate an entire database into a Cloud SQL for SQL Server instance. In that scenario, the simpler and faster approach is to use the built-in import functionality described in [Importing data into Cloud SQL](https://cloud.google.com/sql/docs/sqlserver/import-export/importing).


In [5]:
import csv

rows_added = 0
with pyodbc.connect(CONNECTION_STRING, autocommit=True).cursor() as cursor:
    with open(filename, encoding="ISO-8859-1") as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader) # skip the header row
        for row in csvreader:
            _, name, gender, yearborn, breed, _, zipcode, licensed, *_ = row
            if breed == "Samoyed":
                values = [name, gender, yearborn, zipcode, licensed]
                cursor.execute("INSERT INTO samoyed (name, gender, yearborn, zipcode, licensed) VALUES (?, ?, ?, ?, ?)", values)
                rows_added += 1
                if rows_added >= 20:
                    break

print(f"{rows_added} rows added to the samoyed table")

20 rows added to the samoyed table


## Querying the database
Now that we have a table with data in it, let's try querying our data with a SELECT command.

In [6]:
# Simple query example.
with pyodbc.connect(CONNECTION_STRING).cursor() as cursor:
    cursor.execute("SELECT * FROM samoyed ORDER by zipcode")
    for row in cursor:
        print(f"zipcode: {row.zipcode}, gender: {row.gender}, name: {row.name}")

zipcode: 10009, gender: M, name: KAI
zipcode: 10009, gender: F, name: LAIKA
zipcode: 10009, gender: F, name: YUKI
zipcode: 10013, gender: M, name: OLIVER
zipcode: 10013, gender: M, name: BEAR
zipcode: 10013, gender: F, name: POPPY
zipcode: 10016, gender: M, name: LUX
zipcode: 10023, gender: F, name: BALTI
zipcode: 10023, gender: F, name: HATTIE
zipcode: 10024, gender: F, name: KIKI
zipcode: 10025, gender: M, name: BARON
zipcode: 10025, gender: M, name: BISCUIT
zipcode: 10025, gender: F, name: ALLIE
zipcode: 10065, gender: F, name: VARESSA
zipcode: 10128, gender: M, name: SAMMY
zipcode: 10453, gender: F, name: MARSHMELLO
zipcode: 11207, gender: M, name: SAM
zipcode: 11212, gender: M, name: SAMSON
zipcode: 11228, gender: M, name: LUKA
zipcode: 11234, gender: M, name: PADDY


As shown above, the results of a ```SELECT``` query can be accessed by simply iterating through the cursor itself. There are also several convenience methods to make it easy to work with result sets, such as ```cursor.fetchone()``` or ```cursor.fetchall()``` &mdash; see [Select Basics](https://github.com/mkleehammer/pyodbc/wiki/Getting-started#select-basics) in the pyodbc documentation.

Note that we don't have the optional ```automcommit=True``` connection parameter in this snippet. Since we're just reading data with a SELECT, we don't need to worry about committing transactions.

## Protecting from SQL injection attacks
This has been a quick intro to pyodbc basics, so I've tried to keep things as simple and clear as possible. That means no error handling, and we've also not bothered to [parameterize](https://github.com/mkleehammer/pyodbc/wiki/Getting-started#parameters) our SQL commands.

But if you're working with any form of user inputs or external data sources for the content of your SQL command,  **parameterize your SQL commands to prevent SQL injection attacks!** Don't let [Bobby Tables](https://xkcd.com/327/) mess with your data.

ODBC supports the use of a ```?``` placeholder in a SQL command, which will be replaced at runtime by the value of a separate argument passed to the ```cursor.execute()```. The snippet below shows how to use this feature to run a parameterized query that is protected from SQL injection attacks. For more information see the [pyodbc documentation](https://github.com/mkleehammer/pyodbc/wiki/Getting-started#parameters).

In [8]:
# Example of parameterized query to prevent SQL injection attacks.

# This value is hard-coded, but imagine a scenario where we had a form on a web page
# where a person can enter the ZIPCODE to be queried. In that case, we need to
# use parameterization to protect from the person entering a "zip code" that
# includes a SQL command..
ZIPCODE = "10025"

with pyodbc.connect(CONNECTION_STRING).cursor() as cursor:
    cursor.execute(f"SELECT * FROM samoyed WHERE zipcode = ? ORDER by zipcode", ZIPCODE)
    for row in cursor:
        print(f"zipcode: {row.zipcode}, gender: {row.gender}, name: {row.name}")

zipcode: 10025, gender: M, name: BARON
zipcode: 10025, gender: M, name: BISCUIT
zipcode: 10025, gender: F, name: ALLIE


## Exporting to a Pandas dataframe

If you're a data scientist accustomed to working in Pandas, you'll probably want to get the results of your SQL queries into Pandas dataframes. Pandas provides a ```read_sql()``` function for this, which takes a SQL query and a database connection and returns a dataframe containing the results of the query. The snippet below shows how to use it.

In [18]:
# Example of how to put the results of a SQL SELECT into a Pandas dataframe.
import pandas as pd

database = "testdb" # For this example, we'll connect to the database we created earlier.
CONNECTION_STRING = f"DRIVER={{{config.driver}}};SERVER={config.server};DATABASE={database};UID={config.sql_user};PWD={config.sql_password}"

with pyodbc.connect(CONNECTION_STRING) as cnxn:
    df = pd.read_sql("SELECT * FROM samoyed;", cnxn)
    print(df.head())


         name gender yearborn zipcode    licensed
0        LUKA      M     2009   11228  2014-12-15
1        BEAR      M     2014   10013  2015-01-12
2     VARESSA      F     2014   10065  2015-02-27
3       POPPY      F     2012   10013  2015-03-03
4  MARSHMELLO      F     2004   10453  2015-03-27


## Conclusions
in this notebook, we've learned how to configure a pyodobc connection to a Cloud SQL for SQL Server instance, and then we created a database and table, added some data from a public dataset, and did a few queries. We also learned how to protect our code from SQL injection attacks, and how to get SQL data into Pandas dataframes.

There is much more to pyodbc, of course. The documentation on the [pyodbc wiki](https://github.com/mkleehammer/pyodbc/wiki) is excellent, so that's a great place to start if you want to learn more!